# 分枝限定法

In [7]:
from logging import basicConfig, getLogger, DEBUG, WARNING

basicConfig(level=DEBUG if "get_ipython" in globals() else WARNING)


def sequential_knapsack(m: int, items: list[tuple[int, int]]) -> float:
    logger = getLogger('sequential_knapsack')
    # itemsはコスパが良い順、かつ利用可能なアイテムだけとする
    logger.debug(f"{items=}")
    total_value = 0
    current_space = m
    for value, weight in items:
        if weight <= current_space:
            total_value += value
            current_space -= weight
            logger.debug(f"{items=}, {current_space=}, {total_value=}")
        else:
            total_value += value / weight * current_space
            current_space = 0
        
        if current_space == 0:
            break

    return total_value


def knapsack_by_branch_and_bound(n: int, m: int, items: list[tuple[int, int]]) -> int:
    logger = getLogger('knapsack_by_branch_and_bound')
    # 限定操作のための指標として、最適値の上界を求める
    # 緩和問題として連続ナップザック問題を解く。連続ナップザック問題 = 荷物が液体で、小数点で切り分けられるとする
    sorted_items = sorted(items, key=lambda item: item[0] / item[1], reverse=True)
    global_upper_bound = sequential_knapsack(m, sorted_items)
    logger.debug(f"{sorted_items=}, {global_upper_bound=}")

    def branch_and_bound(mask: tuple[bool]) -> int:
        logger = getLogger('branch_and_bound')
        logger.debug(f"{mask=}")
        if len(mask) == len(sorted_items):
            value = sum([item[0] for item, b in zip(sorted_items, mask) if b])
            weight = sum([item[1] for item, b in zip(sorted_items, mask) if b])
            if weight <= m:
                logger.debug(f"{mask=}, {value=}, {weight=}")
                return value
            else:
                logger.debug(f"{mask=}, {value=}, {weight=}")
                return 0

        # 荷物を詰め込む場合
        # ※ 今回は省略したが、詰め込む荷物単体で耐荷重をオーバーしていたら探索を打ち切ってよい
        true_case_value = branch_and_bound(mask + [True])

        # 部分問題の解が最適解であれば、探索全体を打ち切る
        if true_case_value == global_upper_bound:
            logger.debug(f"部分問題の解が最適解: {true_case_value=}, {global_upper_bound=}")
            return true_case_value
        
        # 荷物を詰め込まない場合
        false_case_possible_pattern = mask + [False] + [True] * (n - len(mask))
        false_case_upper_bound = sequential_knapsack(m, [item for item, b in zip(sorted_items, false_case_possible_pattern) if b])

        if false_case_upper_bound <= true_case_value:
            logger.debug(f"暫定解より良い解を得られない: {true_case_value=}, {false_case_upper_bound=}")
            return true_case_value

        false_case_value = branch_and_bound(mask + [False])
        logger.debug(f"両方の分岐を計算した: {true_case_value=}, {false_case_value=}")
        return max(true_case_value, false_case_value)

    return branch_and_bound([])


In [8]:
expected = 13
actual = knapsack_by_branch_and_bound(4, 5, [(4, 2), (5, 2), (2, 1), (8, 3)])
assert expected == actual, f"{expected=}, {actual=}"

DEBUG:sequential_knapsack:items=[(8, 3), (5, 2), (4, 2), (2, 1)]
DEBUG:sequential_knapsack:items=[(8, 3), (5, 2), (4, 2), (2, 1)], current_space=2, total_value=8
DEBUG:sequential_knapsack:items=[(8, 3), (5, 2), (4, 2), (2, 1)], current_space=0, total_value=13
DEBUG:knapsack_by_branch_and_bound:sorted_items=[(8, 3), (5, 2), (4, 2), (2, 1)], global_upper_bound=13
DEBUG:branch_and_bound:mask=[]
DEBUG:branch_and_bound:mask=[True]
DEBUG:branch_and_bound:mask=[True, True]
DEBUG:branch_and_bound:mask=[True, True, True]
DEBUG:branch_and_bound:mask=[True, True, True, True]
DEBUG:branch_and_bound:mask=[True, True, True, True], value=19, weight=8
DEBUG:sequential_knapsack:items=[(8, 3), (5, 2), (4, 2)]
DEBUG:sequential_knapsack:items=[(8, 3), (5, 2), (4, 2)], current_space=2, total_value=8
DEBUG:sequential_knapsack:items=[(8, 3), (5, 2), (4, 2)], current_space=0, total_value=13
DEBUG:branch_and_bound:mask=[True, True, True, False]
DEBUG:branch_and_bound:mask=[True, True, True, False], value=17, w

In [9]:
expected = 26
actual = knapsack_by_branch_and_bound(4, 10, [(25, 8), (9, 3), (11, 4), (17, 7)])
assert expected == actual, f"{expected=}, {actual=}"

DEBUG:sequential_knapsack:items=[(25, 8), (9, 3), (11, 4), (17, 7)]
DEBUG:sequential_knapsack:items=[(25, 8), (9, 3), (11, 4), (17, 7)], current_space=2, total_value=25
DEBUG:knapsack_by_branch_and_bound:sorted_items=[(25, 8), (9, 3), (11, 4), (17, 7)], global_upper_bound=31.0
DEBUG:branch_and_bound:mask=[]
DEBUG:branch_and_bound:mask=[True]
DEBUG:branch_and_bound:mask=[True, True]
DEBUG:branch_and_bound:mask=[True, True, True]
DEBUG:branch_and_bound:mask=[True, True, True, True]
DEBUG:branch_and_bound:mask=[True, True, True, True], value=62, weight=22
DEBUG:sequential_knapsack:items=[(25, 8), (9, 3), (11, 4)]
DEBUG:sequential_knapsack:items=[(25, 8), (9, 3), (11, 4)], current_space=2, total_value=25
DEBUG:branch_and_bound:mask=[True, True, True, False]
DEBUG:branch_and_bound:mask=[True, True, True, False], value=45, weight=15
DEBUG:branch_and_bound:両方の分岐を計算した: true_case_value=0, false_case_value=0
DEBUG:sequential_knapsack:items=[(25, 8), (9, 3), (17, 7)]
DEBUG:sequential_knapsack:ite